# Main

In [4]:
import pandas as pd
import math
import openpyxl
from scipy import interpolate

In [5]:
# Definition of constants and variables

# Some common physical constants, mainly used to calculate `maxFlux`
## (the upper bound of energy flux received by a planet that allows it
## to be habitable according to Pierrehumbert (2015))

BIGG       = 6.67428e-11       # Gravitational constant
PI         = 3.1415926535
A          = 0.7344            # Pierrehumbert's Constant
SB         = 5.670373e-8       # Stefan-Boltzmann Constant
LH2O       = 2.425e6           # Latent Heat Capacity of Water
RGAS       = 461.5             # Universal Gas Constant
PLINE      = 1e4               
PREF       = 610.616           # Reference Pressure
TREF       = 273.13            # Reference Temperature
K0         = 0.055             # A constant in Runaway Greenhouse calculation


# Boundaries for albedo values:
ALBMINELSE = 0.05              # General lower bound
ALBMAXELSE = 0.8               # General upper bound
ALBMING    = 0.25              # Lower bound for planets orbiting G-type stars
ALBMAXM    = 0.35              # Upper bound for planets orbiting M-type stars


# The counterpart to maxFlux
## (unlike maxFlux, MINFLUX is a constant that does not depend on a planet's properties)
MINFLUX    = 67

# Definitions of units of measurement,
## mainly used to convert [exoplanet.org](exoplanet.org) data into SI units:
MEARTH     = 5.972186e24       # Earth mass in kilograms
REARTH     = 6378100           # Earth's radius in meters
S0         = 1362              # Solar constant in watts per square meter
MSUN       = 1.988416e30       # Solar mass in kilograms
RSUN       = 6.957e8           # Solar radius in meters
LSUN       = 3.828e26          # Solar luminosity in watts
RJUP       = 7.1492e7          # Jovian radius in meters
MJUP       = 1.8982e27         # Jovian mass in kilograms
AU         = 1.496e11          # The astronomical unit in meters

In [6]:
# Definition of functions

# 1) A function to calculate the probability distribution of orbital eccentricity
def pofe(ecc,mu,sigma):
    return ((sigma*math.sqrt(2*math.pi))**(-1))*math.exp(-(((ecc-mu)**2)/(2*sigma**2)))/1000

# 2) A function to calculate the probability of a planet's terrestriality
def fp_ter(mPlanet,rPlanet,exoName):
    # Convert the unit to Earth's masses and radii
    mPlanet = mPlanet/MEARTH 
    rPlanet = rPlanet/REARTH
    
    # Calculate mu1
    ## Initialize mu1 value to 0
    mu1 = 0.0       
    
    # Initialize temporary variables to hold a mass/radius value
    ## from the (i-1)th row of the ZS table
    mZSimin1 = 0
    rZSimin1 = 0

    # This block iterates through the the 'M-Pure-MgSiO3' column
    #to find the bracket that contains mPlanet value
    for i in rowNum:
        # Initialize temporary variables to hold a mass/radius value
        #from the i-th row of the ZS table
        mZSi = zs.loc[i, "M-PureMgSiO3"]
        rZSi = zs.loc[i, "R-PureMgSiO3"]
        
        # Comparing mPlanet to the current value of mZSi
        if mPlanet == mZSi:
            mu1 = rZSi
            break                          
        elif mPlanet > mZSi:               
            mZSimin1 = mZSi                
            rZSimin1 = rZSi                  
        else: # if mPlanet < mZSi --> we have found the correct bracket
            f = interpolate.interp1d(zs.loc[(i-1):(i), "M-PureMgSiO3"], zs.loc[(i-1):(i), "R-PureMgSiO3"], kind='linear', assume_sorted=True)
            mu1 = f(mPlanet)
            break

    # Calculate mu2
    mu2 = 0.0
    mZSimin1 = 0
    rZSimin1 = 0
    for i in rowNum:
        mZSi = zs.loc[i, "M-MgSiO3-H2O-5050"]
        rZSi = zs.loc[i, "R-MgSiO3-H2O-5050"]
        if mPlanet == mZSi:
            mu2 = rZSi
            break
        elif mPlanet > mZSi:
            mZSimin1 = mZSi
            rZSimin1 = rZSi
        else: 
            f = interpolate.interp1d(zs.loc[(i-1):(i), "M-MgSiO3-H2O-5050"], zs.loc[(i-1):(i), "R-MgSiO3-H2O-5050"], kind='linear', assume_sorted=True)
            mu2 = f(mPlanet)
            break

    # Calculate sigma1
    sigma1 = (mu2-mu1)/3
    
    # Calculate the terrestrial probability
    p_ter = 0
    if rPlanet <= mu1:
        p_ter = 1
    elif rPlanet >= mu2:
        p_ter = 0
    else: # uses a pseudo-gaussian function
        p_ter = math.exp(-(0.5)*((rPlanet-mu1)/sigma1)**2)
    return p_ter

In [7]:
# Data input
## Import exoplanet data from a CSV file into a pandas dataframe
exo = pd.read_csv (r'nasa_exoplanets.csv', low_memory=False)

# Set the column with the header NAME to be used as an index to identify row 
exo = exo.set_index("pl_name", drop = False)

# Extract names of planets as a list (to be used as a calling list)
exoList = pd.DataFrame(exo, columns=['pl_name'])
exoList = exoList['pl_name'].values.tolist()

In [8]:
# Zeng-Sasselov boundaries input

## Import CSV of Zeng & Sasselov boundaries
zs = pd.read_csv (r'zsboundaries.csv')

## Set index using the RowNum column
zs = zs.set_index("RowNum", drop = False)

## Extract the column "RowNum" as a list (to be used as a calling list)
rowNum = pd.DataFrame(zs, columns=['RowNum'])
rowNum = rowNum['RowNum'].values.tolist()

In [25]:
# Main subroutine to determine the habitability index value

habIndex = []
habIndexWithName = []
habIndexNotZero = []
i = 1

for exoName in exoList:
    #print(f'{i}) {exoName}')
    i = i + 1
    
    # Extract data of individual planets
    # HOST STAR PROPERTIES
    # Stellar radius (in solar radii)
    rStar = exo.loc[exoName, "st_rad"]
    ## Convert to SI
    rStar = rStar*RSUN
    
    
    # Stellar temperature (in Kelvin)
    teffStar = exo.loc[exoName, "st_teff"]
    
    
    # Stellar luminosity
    luminosity = 4*math.pi*rStar*rStar*SB*teffStar**4
    
    # PLANET PROPERTIES   
    # Planetary radius (in Jovian radii)
    rPlanet = exo.loc[exoName, "pl_radj"]
    ## If R is not available, calculate it from  transit depth
    if math.isnan(rPlanet) == 1:
        depth = exo.loc[exoName, "pl_trandep"]
        if math.isnan(depth) == 1:
            continue
        rPlanet = math.sqrt(depth)*rStar
    ## Convert to SI
    rPlanet = rPlanet*RJUP
    
    # Planetary mass (in Jovian masses)
    mPlanet = exo.loc[exoName, "pl_bmassj"] 
    ## If MASS is not available, calculate it from a common scaling law
    ### from the original HITE
    if math.isnan(mPlanet) == 1:
        if rPlanet/REARTH <= 1:
            mPlanet = ((rPlanet/REARTH)**3.268)*MEARTH
        elif rPlanet/REARTH > 1:
            mPlanet = ((rPlanet/REARTH)**3.65)*MEARTH
    ## Convert to SI
    mPlanet = mPlanet*MJUP
    
    # Surface planet gravity (in SI)
    surfGrav = BIGG*mPlanet/(rPlanet**2)    
    
    
    # ORBITAL PROPERTIES
    
    # Orbital eccentricity
    ecc = exo.loc[exoName, "pl_orbeccen"]
    if math.isnan(ecc) == 1:
        continue

    # Measurement uncertainty of orbital eccentricity    
    ## Upper bound (relative from E)
    eccUpRel = exo.loc[exoName, "pl_orbeccenerr1"]
    ### If measurement uncertainty is not available, assign it as 0.01
    if math.isnan(eccUpRel) == 1 or eccUpRel == 0:
        eccUpRel = 0.01
    ### Upper bound (absolute)
    eccUpper = ecc + eccUpRel
    
    
    ## Lower bound (relative from E)
    eccLowRel = exo.loc[exoName, "pl_orbeccenerr2"]
    ### If measurement uncertainty is not available, assign it as 0.01
    if math.isnan(eccLowRel) == 1:
        if ecc != 0:
            eccLowRel = -0.01
        else:
            eccLowRel = 0
    ### Lower bound (absolute)
    eccLower = ecc + eccLowRel
    #(NASA exo. archive uses negative number for the ecc. lower bound.)
    
    # Orbital semi-major axis (in AU)
    semiAxis = exo.loc[exoName, "pl_orbsmax"]
    ## Convert to SI
    semiAxis = semiAxis*AU
    
    
    # Calculate the upper and lower bounds of F_OLR [...]
    ## that would allow for surface liquid water to exist
    pStar = PREF*math.exp(LH2O/(RGAS*TREF))
    # Upper bound: maximum F_OLR
    maxFlux = A*SB*(LH2O/(RGAS*math.log(pStar*math.sqrt(K0/(2*PLINE*surfGrav)))))**4
    # Lower bound: minimum F_OLR is the constant MINFLUX
    minFlux = MINFLUX
    
    # Probability of the planet being terrestrial
    p_ter = fp_ter(mPlanet,rPlanet,exoName)
        
    
    # Albedo (new)
    ## Boundaries
    albMin = ALBMINELSE
    albMax = ALBMAXELSE
    ## Special conditions
    ### For planets with M-type host star
    #if teffStar >= 2300 and teffStar <=3800:
    #    albMax = ALBMAXM
    ### For planets with G-type host star
    #elif teffStar >= 5370 and teffStar <=5980:
    #    albMin = ALBMING
        
        
    
    # Calculate F_OLR
    ## Albedo increments
    da = 0.01
    ## Eccentricity increments
    de = 0.01
    ## Sum of pofe (probability of eccentricity);
    ### (is used to normalize the index value, later)
    ### Initialized to 0
    pofeSum = 0
    ### Sum of how many instances of F_OLR meets the requirements for
    #### the planet to have surface liquid water. Each instances will then be
    #### multiplied by the probability of its eccentricity (pofe)
    ### Initialized to 0
    habFact = 0
    ### Incoming stellar radiation (instellation)
    flux0 = luminosity/(16*math.pi*semiAxis*semiAxis)

    # Calculate the habitability index
    ## Iterate through the albedo & eccentricity 2D matrix
    a = albMin
    while a < albMax:
        e = eccLower
        while e < eccUpper:
            flux = flux0*(1-a)/math.sqrt(1-e*e)
            pofeSum = pofeSum + pofe(e, ecc, eccUpRel)
            if flux < maxFlux and flux > MINFLUX:
                habFact = habFact + pofe(e, ecc, eccUpRel)
            e = e + de
        a = a + da   
    
    if ecc > 0.8:
        H = 0.0
    elif pofeSum != 0:
        H = (habFact/pofeSum)*p_ter
    else: # in the case of error; might be better to replace this with a throw exception statement
        H = 0.0
    
    habIndex.append(H)
    habIndexWithName.extend([[exoName, H]])
    
    if H > 0:
        habIndexNotZero.extend([[exoName, str(H)]])
    #print(H)
    #print()

In [26]:
dfHabIndexWithName = pd.DataFrame(habIndexWithName,columns=['pl_name','hab_index'])
dfHabIndexNotZero = pd.DataFrame(habIndexNotZero,columns=['pl_name','hab_index'])

print(dfHabIndexNotZero)

with pd.ExcelWriter('results_nasadb.xlsx') as file:
    dfHabIndexNotZero.to_excel(file, sheet_name='HabIndexNotZero')
    dfHabIndexWithName.to_excel(file, sheet_name='HabIndexWithName')


               pl_name             hab_index
0            GJ 1061 b  0.013333333333333332
1            GJ 1061 c    0.5199999999999996
2            GJ 1061 d    0.9066666666666657
3            GJ 1132 c     0.245122339076673
4             GJ 251 b   0.00366417717359088
..                 ...                   ...
75  Teegarden's Star c    0.6133333333333317
76         Wolf 1061 c   0.10309192833328942
77            YZ Cet d   0.14123273046286972
78           tau Cet e  0.011026593958767423
79           tau Cet f  0.009238377535128113

[80 rows x 2 columns]


In [27]:
for i in habIndexNotZero:
    print(i)

['GJ 1061 b', '0.013333333333333332']
['GJ 1061 c', '0.5199999999999996']
['GJ 1061 d', '0.9066666666666657']
['GJ 1132 c', '0.245122339076673']
['GJ 251 b', '0.00366417717359088']
['GJ 273 b', '0.40488194238455266']
['GJ 3293 e', '0.02370462563733535']
['GJ 3323 b', '0.6962361824833894']
['GJ 411 b', '0.032426890128164035']
['GJ 625 b', '0.18274960159194573']
['HD 219134 d', '0.026666666666666658']
['HD 85512 b', '0.015507210621684173']
['K2-3 c', '0.00046109522713008835']
['K2-3 d', '0.21409345359060591']
['K2-72 c', '0.2942803628019286']
['K2-72 e', '0.7825746183089755']
['Kepler-1185 b', '0.053325386211250055']
['Kepler-1229 b', '0.7196551649547821']
['Kepler-138 d', '0.0020398709698343856']
['Kepler-138 e', '0.3507610418428164']
['Kepler-1389 b', '0.011200170972980915']
['Kepler-1410 b', '0.01934087296314226']
['Kepler-1450 b', '0.009610694124363666']
['Kepler-1459 b', '0.07958379064276028']
['Kepler-1544 b', '0.03315578222252954']
['Kepler-1599 b', '0.0029357612019888873']
['Kepl

In [ ]:
# Append the result (planet's name and index value) to a .txt file in the same folder 
with open('out_nasadb.txt','w') as f:
    i = 1
    for a in habIndexWithName:
        if i == 3:
            print(a, file=f)
            i = 1
        else:
            print(a, file=f, end="")
            i += 1

In [ ]:
5233